# QUICKSTART
## 数据集

两个数据集
torch.utils.data.DataLoader 包装了Dataset的迭代器
torch.utils.data.Dataset  包含了样本和相关的标签

TorchText, TorchVision, and TorchAudio都报班了dataset
每个 TorchVision 数据集都包含两个参数：transform 和 target_transform，分别用于修改样本和标签。

In [17]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

#


# 从开放数据集中下载训练数据.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

我们将数据集作为参数传递给 DataLoader。
这在我们的数据集上包装了一个可迭代对象，并支持自动批处理、采样、洗牌和多进程数据加载。
这里我们定义批量大小为 64，即数据加载器迭代中的每个元素将返回一批 64 个特征和标签。

In [18]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## 创建模型

为了在 PyTorch 中定义神经网络，我们创建一个继承自 nn.Module 的类。
我们在 __init__ 函数中定义网络层，并在前向函数中指定数据如何通过网络。
为了加速神经网络中的操作，我们将其转移到 GPU 或 MPS（如果可用）。

In [19]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## 优化模型

为了训练模型，我们需要损失函数（loss function）和优化器（optimizer）

In [20]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

在单个训练循环中，模型对训练数据集（批量输入）进行预测，并反向传播预测误差以调整模型的参数。

In [21]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

我们还根据测试数据集检查模型的性能，以确保它正在学习

In [22]:

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

训练过程需要进行多次迭代（epoch）。
在每个时期，模型都会学习参数以做出更好的预测。 我们打印每个时期模型的准确性和损失； 我们希望看到每个 epoch 的准确率都会提高，损失也会减少。

In [23]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.298187  [   64/60000]
loss: 2.287460  [ 6464/60000]
loss: 2.268719  [12864/60000]
loss: 2.262282  [19264/60000]
loss: 2.240592  [25664/60000]
loss: 2.227759  [32064/60000]
loss: 2.224696  [38464/60000]
loss: 2.199257  [44864/60000]
loss: 2.195795  [51264/60000]
loss: 2.160345  [57664/60000]
Test Error: 
 Accuracy: 50.0%, Avg loss: 2.156364 

Epoch 2
-------------------------------
loss: 2.163530  [   64/60000]
loss: 2.153956  [ 6464/60000]
loss: 2.099411  [12864/60000]
loss: 2.114296  [19264/60000]
loss: 2.057936  [25664/60000]
loss: 2.013321  [32064/60000]
loss: 2.033995  [38464/60000]
loss: 1.961403  [44864/60000]
loss: 1.960019  [51264/60000]
loss: 1.885435  [57664/60000]
Test Error: 
 Accuracy: 56.2%, Avg loss: 1.887825 

Epoch 3
-------------------------------
loss: 1.911684  [   64/60000]
loss: 1.881137  [ 6464/60000]
loss: 1.773952  [12864/60000]
loss: 1.815859  [19264/60000]
loss: 1.700500  [25664/60000]
loss: 1.662377  [32064/600